In [1]:
import os
import sys
sys.path.append('../../')

import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import xgboost as xgb
import joblib

import core.config as conf

## Preprocessing

In [2]:
path = conf.dataset_path

In [3]:
book_df = pd.read_csv(f'{path}'+'books.csv')

In [4]:
rating_df = pd.read_csv(f'{path}' + 'binary_ratings.csv')

In [5]:
df = pd.merge(book_df, rating_df, how='right', on='book_id')

In [6]:
user_mean_rating = rating_df['binary_rating'].groupby(rating_df['user_id']).mean().to_frame()
user_mean_rating = user_mean_rating.reset_index()

In [7]:
user_mean_rating.columns = ["user_id", "user_mean_rating"]

In [8]:
df = pd.merge(df, user_mean_rating, how='left', on = 'user_id')

In [9]:
df = df.dropna()

In [10]:
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df[['language_code']])

In [11]:
df[one_hot.categories_[0]] = encoded.toarray()

In [12]:
df = df.drop(conf.dont_used_features, axis = 1)

## Recommend top 10 books

In [13]:
user_id = 314

In [14]:
df[df['user_id'] == 314]

,book_id,books_count,original_publication_year,average_rating,ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,user_id,binary_rating,user_mean_rating,en-CA,en-GB,en-US,eng,fre,spa
0,1,275.0,2005.0,4.54,1678823.0,27520.0,7308.0,21516.0,136333.0,459028.0,1161491.0,314,1,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
200,3,491.0,1997.0,4.44,4602479.0,75867.0,75504.0,101676.0,455024.0,1156318.0,3011543.0,314,0,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
400,5,376.0,1999.0,4.53,1832823.0,36099.0,6716.0,20413.0,166129.0,509447.0,1266670.0,314,1,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
501,6,332.0,2000.0,4.53,1753043.0,31084.0,6676.0,20210.0,151785.0,494926.0,1195045.0,314,1,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
2800,29,28.0,1990.0,3.95,24133.0,1993.0,333.0,1280.0,6050.0,10985.0,8303.0,314,0,0.574586,0.0,0.0,1.0,0.0,0.0,0.0
2900,30,74.0,1973.0,4.59,90907.0,1565.0,826.0,1281.0,5819.0,19982.0,66440.0,314,1,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
9700,98,45.0,1989.0,3.86,10807.0,784.0,359.0,816.0,3301.0,4586.0,4185.0,314,0,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
10400,105,72.0,1985.0,3.89,34822.0,552.0,676.0,2612.0,9170.0,12496.0,12201.0,314,0,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
10501,106,140.0,1969.0,3.86,82580.0,2283.0,1026.0,5333.0,23541.0,36242.0,24290.0,314,1,0.574586,0.0,0.0,0.0,1.0,0.0,0.0
34200,343,364.0,1985.0,4.00,178135.0,10057.0,5659.0,12862.0,41804.0,78658.0,83771.0,314,0,0.574586,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
user_read_books = set(df[df['user_id'] == 314]['book_id'].values)

In [16]:
user_read_books

{1, 3, 5, 6, 29, 30, 98, 105, 106, 343, 348, 428, 760, 976, 2054, 6174}

In [17]:
book_lst = set(book_df['book_id'].values)

In [18]:
book_lst

{1,
 2,
 3,
 32769,
 5,
 6,
 3407877,
 2195464,
 196613,
 8,
 11,
 10,
 13,
 7766027,
 3080212,
 21,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 6357022,
 10813469,
 33,
 34,
 1736739,
 36,
 294951,
 32807,
 426025,
 10813480,
 32812,
 426030,
 50,
 131123,
 294963,
 229432,
 32827,
 32829,
 32831,
 688191,
 26771521,
 67,
 1474628,
 65605,
 1048645,
 15728721,
 262228,
 262229,
 16056408,
 93,
 65633,
 98,
 98405,
 65641,
 106,
 105,
 721003,
 17465453,
 98413,
 21856367,
 112,
 360562,
 721012,
 7864437,
 117,
 119,
 122,
 98427,
 15827066,
 17465470,
 10846336,
 6553733,
 163977,
 8519822,
 65684,
 20316309,
 6422680,
 589979,
 196764,
 32929,
 327847,
 17465515,
 295086,
 16122034,
 11337912,
 15990969,
 7831742,
 11010251,
 32979,
 491735,
 1081560,
 32987,
 295132,
 17334495,
 231,
 131304,
 7307503,
 24903919,
 655601,
 24019187,
 249,
 250,
 9666812,
 295169,
 12812550,
 264,
 98573,
 13041934,
 6390031,
 275,
 15728921,
 12288282,
 262430,
 131359,
 23363874,
 11600163,
 8683812,
 291

In [19]:
user_not_read_books = book_lst - user_read_books

In [20]:
rec_df = book_df

In [21]:
df.columns

Index(['book_id', 'books_count', 'original_publication_year', 'average_rating',
       'ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2',
       'ratings_3', 'ratings_4', 'ratings_5', 'user_id', 'binary_rating',
       'user_mean_rating', 'en-CA', 'en-GB', 'en-US', 'eng', 'fre', 'spa'],
      dtype='object')

In [22]:
rec_df.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [23]:
user_not_read_books = book_lst - user_read_books

In [24]:
rec_df['binary_rating'] = -1

In [25]:
rec_df['user_mean_rating'] = df[df['user_id'] == user_id]['user_mean_rating'][0]

In [26]:
rec_df = rec_df.dropna()

In [27]:
language_lst = ['en-CA', 'en-GB', 'en-US', 'eng', 'fre', 'spa']

In [28]:
rec_df = rec_df.drop(rec_df[~rec_df['language_code'].isin(language_lst)].index)

In [29]:
rec_df['language_code'].unique()

array(['eng', 'en-US', 'en-CA', 'spa', 'en-GB', 'fre'], dtype=object)

In [30]:
encoded = one_hot.transform(rec_df[['language_code']])

In [31]:
rec_df[one_hot.categories_[0]] = encoded.toarray()

In [42]:
rec_df = rec_df.drop(conf.dont_used_features, axis = 1)

In [54]:
col_x = rec_df.columns
col_x = col_x.drop('binary_rating')

In [55]:
book_df.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url', 'user_id', 'binary_rating', 'user_mean_rating'],
      dtype='object')

In [56]:
rec_df.columns

Index(['book_id', 'books_count', 'original_publication_year', 'average_rating',
       'ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2',
       'ratings_3', 'ratings_4', 'ratings_5', 'user_id', 'binary_rating',
       'user_mean_rating', 'en-CA', 'en-GB', 'en-US', 'eng', 'fre', 'spa'],
      dtype='object')

In [57]:
dpred = xgb.DMatrix(data = rec_df[col_x])

In [37]:
model = joblib.load(conf.model_path+conf.model_name)

In [59]:
y_pred = model.predict(dpred)

In [92]:
recommend_lst = pd.DataFrame(rec_df['book_id'])

In [102]:
recommend_lst = pd.merge(recommend_lst, book_df[['book_id', 'title']], on= 'book_id')

In [103]:
recommend_lst['pred'] = y_pred

In [117]:
recommend_lst.sort_values(by=['pred'], ascending=False, axis=0).head(10)

,book_id,pred,title
2492,5344,0.867610,Hard Times
26,1,0.863471,Harry Potter and the Half-Blood Prince (Harry ...
5557,250,0.860365,Tropic of Capricorn
52,11,0.853773,The Hitchhiker's Guide to the Galaxy (Hitchhik...
5205,2872,0.845988,Falling Angels
837,870,0.840194,"Fullmetal Alchemist, Vol. 1 (Fullmetal Alchemi..."
4268,359,0.835942,"The Salmon of Doubt (Dirk Gently, #3)"
6998,4599,0.831202,Carter Beats the Devil
141,2865,0.829108,Girl with a Pearl Earring
4866,5289,0.828172,Complete Works of Oscar Wilde


In [120]:
rec_top10 = recommend_lst.sort_values(by=['pred'], ascending=False, axis=0)['book_id'].values[:10]

In [122]:
for idx, book in enumerate(rec_top10) :
    print(f"rank {idx+1} : ", book)

rank 1 :  5344
rank 2 :  1
rank 3 :  250
rank 4 :  11
rank 5 :  2872
rank 6 :  870
rank 7 :  359
rank 8 :  4599
rank 9 :  2865
rank 10 :  5289


In [114]:
user_read_books

{1, 3, 5, 6, 29, 30, 98, 105, 106, 343, 348, 428, 760, 976, 2054, 6174}

## Popular top10

In [15]:
ratings_df = pd.read_csv(conf.dataset_path + 'binary_ratings.csv')

In [17]:
book_df = pd.read_csv(conf.dataset_path + 'books.csv')

In [19]:
book_ratings = ratings_df['binary_rating'].groupby(ratings_df['book_id']).sum().to_frame()

In [22]:
book_ratings = pd.merge(book_ratings, book_df[['book_id', 'title']], on = 'book_id')

In [23]:
book_ratings.sort_values(by=['binary_rating'], ascending=False)

,book_id,binary_rating,title
769,9566,97,Still Life with Woodpecker
291,3885,95,The Taste of Home Cookbook
354,4708,94,The Beautiful and Damned
206,2767,93,A People's History of the United States
771,9569,92,Villa Incognito
...,...,...,...
728,8960,26,"Vampire Mountain (Cirque Du Freak, #4)"
789,9762,23,Loving What Is: Four Questions That Can Change...
579,6819,22,Ghostwritten
304,4009,21,Nine Stories


In [27]:
rec_pop_top10 = book_ratings.sort_values(by=['binary_rating'], ascending=False, axis=0)['title'].values[:10]

In [29]:
for idx, book in enumerate(rec_pop_top10) :
    print(f"rank {idx+1} : ", book)

rank 1 :  Still Life with Woodpecker
rank 2 :  The Taste of Home Cookbook
rank 3 :  The Beautiful and Damned
rank 4 :  A People's History of the United States
rank 5 :  Villa Incognito
rank 6 :  The Lost Boy (Dave Pelzer #2)
rank 7 :  Franny and Zooey
rank 8 :  Girl with a Pearl Earring
rank 9 :  Hard Times
rank 10 :  The Last Juror
